In [ ]:
# Carregando bibliotecas de uso geral
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [ ]:
# Carregando conjunto de treinamento
df_train = pd.read_csv('bases/mnist-in-csv/mnist_train.csv', sep=',')
X_train  = df_train.values[:, 1:]
Y_train  = df_train.values[:, 0]

# Carregando conjunto de testes
df_test = pd.read_csv('bases/mnist-in-csv/mnist_test.csv', sep=',')
X_test  = df_test.values[:, 1:]
Y_test  = df_test.values[:, 0]

In [ ]:
# Vamos ver o que temos no conjunto...
plt.figure(figsize=(20,4))
for index, (image, label) in enumerate(zip(X_train[0:5], Y_train[0:5])):
    plt.subplot(1, 5, index + 1)
    plt.imshow(np.reshape(image, (28,28)), cmap=plt.cm.gray)
    plt.title('Treinamento: %i\n' % label, fontsize = 20)

In [ ]:
# Algumas amostras do dígito 9 (pode escolher o dígito abaixo)
digito = 9
fig, ax = plt.subplots(nrows=5, ncols=5, figsize=(10,10), sharex=True, sharey=True)
ax = ax.flatten()
for i in range(25):
    img = X_train[Y_train == digito][i].reshape(28, 28)
    ax[i].imshow(img, cmap=plt.cm.gray)
    
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.show()

In [ ]:
#
# Normalizando dados (por se tratar de imagens a normalização é um pouco diferente)
#
# No livro "Python Machine Learning" do Raschka tem uma explicação sobre isso.
#
Xz_train = ((X_train / 255.0) - 0.5)*2
Xz_test  = ((X_test / 255.0) - 0.5)*2

In [ ]:
#
# O primeiro classificador que vamos utilizar é uma regressão logística
# vamos otimizar os hiperparâmetros com GridSearch
#
from sklearn.model_selection import ShuffleSplit

# Criando instância da regressão logística
clf = LogisticRegression(solver='sag', 
                         max_iter=1000, 
                         multi_class='ovr')

# Otimizando hiperparâmetros com GridSearch (vamos variar o parâmetro de regularização C)
grid = [{'C':[0.1, 1, 100, 1000]}]

#
# Grid de busca (vamos remover a validação cruzada para acelerar o treinamento)
#
cv = ShuffleSplit(test_size=0.20, n_splits=1, random_state=0)
gs = GridSearchCV(clf, grid, n_jobs=-1, cv=cv)
gs.fit(Xz_train, Y_train)

#
# Imprimindo melhor modelo encontrado
#
print('Melhor score de validação %.3f'% (gs.best_score_*100), '%')
print('Melhor score de teste %.3f'% (gs.score(Xz_test, Y_test)*100), '%')
print('Melhores hiperparâmetros', gs.best_params_)

In [ ]:
# Melhor modelo encontrado
clf = LogisticRegression(solver='sag', 
                         max_iter=1000, 
                         multi_class='ovr',
                         C=1)

# Treinando melhor modelo
clf.fit(Xz_train, Y_train)

# Plotando acurácias
print('Melhor score de treinamento %.3f'% (clf.score(Xz_train, Y_train)*100), '%')
print('Melhor score de teste %.3f'% (clf.score(Xz_test, Y_test)*100), '%')